In [ ]:
from omnipy import Model

In [ ]:
from pydantic import BaseModel, Field, PrivateAttr, validator
from pydantic.fields import UndefinedType, Undefined
from pydantic.generics import GenericModel
from typing import Generic
from typing_extensions import TypedDict, TypeVar, TypeVarTuple, Unpack
ModelT = TypeVarTuple('ModelT')
ModelT.__bound__ = Model  # Pydantic v1 hack
BlueprintT = TypeVar('BlueprintT')

class NoBlueprintType(object):
    def __repr__(self):
        return 'NoBlueprint'


NoBlueprint = NoBlueprintType()

ModelsT = TypeVarTuple('ModelTs')
ModelsT.__bound__ = Model  # Pydantic v1 hack

In [ ]:
class A(Generic[Unpack[ModelsT]]):
    @classmethod
    def __class_getitem__(cls, items: Unpack[ModelsT]):
        print(f'Creating A with items: {items}')
        created_cls = super().__class_getitem__(items)
        # created_cls.models = items
        return created_cls

class B(A[ModelsT], Generic[Unpack[ModelsT]]):
    ...




In [ ]:
b= B[Model[int], Model[str]]()
b

In [ ]:
from typing import ClassVar
from typing_extensions import TypedDict, TypeVar, TypeVarTuple, Unpack
from collections import UserDict
ModelsT = TypeVarTuple('ModelTs')
ModelsT.__bound__ = Model  # Pydantic v1 hack
BlueprintHolderT = TypeVar('BlueprintHolderT', bound='BlueprintHolder')
BlueprintT = TypeVar('BlueprintT', bound='BluePrint')

class BluePrint(TypedDict):
    ...



class BlueprintHolder(GenericModel, Generic[ModelsT]):
    __root__: BluePrint = Field(default_factory=lambda: Model)
    @classmethod
    def __class_getitem__(cls, items: Unpack[ModelsT]):
        created_cls = super().__class_getitem__(items)
        created_cls.models = items
        return created_cls

    
    @classmethod
    def __init_subclass__(cls, blueprint:type[BlueprintT] | None = None, **kwargs):
        print(f'Initializing subclass {cls.__name__} with blueprint={blueprint}')
        print(f'cls: {cls}, type(blueprint): {type(blueprint)}')
        cls._blueprint = blueprint
        super().__init_subclass__(**kwargs)


    _models: ClassVar[type[ModelsT]] = PrivateAttr()
    _blueprint: ClassVar[type[BlueprintT]] = PrivateAttr()




class Dataset(UserDict, Generic[BlueprintHolderT]):
    def __init__(self, dict=None, /, **kwargs):
        super().__init__(dict, **kwargs)

In [ ]:
MyBlueprint = Blueprint[Model[int], Model[str]]

In [ ]:
d = MyBlueprint()
d.__class__

In [ ]:
from types import NoneType
from pydantic.fields import ModelField
from typing import ClassVar, get_args, cast, Union, Type, Any, Tuple

class Blueprint(TypedDict):
    ...

BlueprintT = TypeVar('BlueprintT', bound=Blueprint)




class MyDataset(GenericModel, Generic[Unpack[ModelsT], BlueprintT]):
    @classmethod
    def __class_getitem__(cls, items: Unpack[ModelsT], blueprint: type[BlueprintT]| None=None) -> type['MyDataset']:
        blueprint: type[BlueprintT] = items[-1]
        items = items[:-1] if len(items) > 1 else ()
        print(f'Creating MyDataset with blueprint: {blueprint}, items: {items}')
        args = []
        # args = []
        if isinstance(items, tuple):
            print(f'items: {items}')
            args += [Union[items]]  # type: ignore

        else:
            args += [items]
        if blueprint is not None:
            args += [blueprint]
        if len(args) > 0:
            print(f'args: {args}, type(args): {type(args)}')
            created_cls = super().__class_getitem__(tuple(args))
        else:
            raise ValueError('At least one model type must be provided')
        # if len(items) > 1:
        print(f'created_cls: {created_cls}, type(created_cls): {type(created_cls)}')
        print(f'items: {items}, type(items): {type(items)}')
        created_cls._model = tuple(items)
        return created_cls


    _blueprint: ClassVar[type[Blueprint] | None] = PrivateAttr()
    _model: ClassVar[type[Model]] = PrivateAttr()
    # data: dict[str, ModelsT] = Field(default_factory=lambda: dict)
    data: BlueprintT = Field(default_factory=lambda: Blueprint)

    def __init_subclass__(cls, blueprint:type[BlueprintT] | None = None, **kwargs):
        print(f'Initializing subclass {cls.__name__} with blueprint={blueprint}')
        print(f'cls: {cls}, type(blueprint): {type(blueprint)}')
        cls._blueprint = blueprint


    class Config:
        validate_assignment = True
        arbitrary_types_allowed = True
    #
    # def __new__(cls, *args, **kwargs):
    #     print(f'Creating instance of {cls.__name__} with args={args}, kwargs={kwargs}')
    #
    #     print(f'cls: {cls}, get_args(cls): {get_args(cls)}')
    #     args = []
    #     args += list(cls._model)
    #     if cls._blueprint is None:
    #         args += [None]
    #         subcls = MyDictDataset[tuple(args)]
    #     else:
    #         args += [cls._blueprint]
    #         subcls = MyBlueprintDataset[tuple(args)]
    #     print(f'cls: {cls}, subcls: {subcls}')
    #     print(f'subcls: {subcls}, type(subcls): {type(subcls)}')
    #     instance = subcls.__new__(subcls, **kwargs)
    #     return instance


# class MyDictDataset(MyDataset[ModelsT], Generic[Unpack[ModelsT]]):


    # def __new__(cls, *args, **kwargs):
    #     return GenericModel.__new__(cls)


# class MyBlueprintDataset(MyDataset[ModelsT, BlueprintT], Generic[Unpack[ModelsT], BlueprintT], blueprint = Blueprint):
#     data: BlueprintT = Field(default_factory=lambda: Blueprint)

    # @classmethod
    # def __class_getitem__(cls, items: Unpack[ModelsT]):
    #     print(f'Creating MyBlueprintDataset with items: {items}')
    #     return super().__class_getitem__(items[:-1])  # Exclude the last item which is BlueprintT

    # def __new__(cls, *args, **kwargs):
    #     return GenericModel.__new__(cls)


    #
    # def __init__(self, data: Blueprint | dict[str, Model] = None, **kwargs):
    #     print(f'data: {data}')
    #     if data is None:
    #         data = self._get_blueprint()
    #     blueprint_cls = self._blueprint
    #     print(f'blueprint_cls: {self, blueprint_cls}')
    #     if blueprint_cls is None:
    #         blueprint_cls = Union[self._model]
    #     print(f"self.__fields__['data'].type_: {self.__fields__['data'].type_}")
    #     self.__fields__['data'].type_ = blueprint_cls
    #     print(f"self.__fields__['data'].type_: {self.__fields__['data'].type_}")
    #     self.__fields__['data'].default_factory = blueprint_cls
    #     my_kwargs = kwargs.copy()
    #     my_kwargs['data'] = data
    #     print(f'my_kwargs: {my_kwargs}')
    #     super().__init__(**my_kwargs)
    #     print(f'self.data: {self.data}')

    @validator('data')
    def validate_data(cls, value):
        print(f'cls._model: {cls._model}')
        for key, val in value.items():
            print(f'Validating value: {val}')
            print(cls)
            for model in cls._model(val):  # This will raise an error if value is not of type cls._model

    # @validator('val', pre=True, always=True)
    # def val_is_not_none(cls, value:ModelT | UndefinedType | None) -> ModelT | UndefinedType:
    #     # if value is None:
    #     #     raise ValueError('val cannot be None')
    #     return value

In [ ]:
asd = MyDictDataset[Model[float]](data=dict(number='1234.1', text=123.0))
asd

In [ ]:
class MyBlueprint(Blueprint, total=False):
    number: Model[list[int]]
    text: Model[list[str]]

class MyNewBlueprintDataset(MyBlueprintDataset[Model[int], Model[list], MyBlueprint], blueprint=MyBlueprint):
    pass

MyBlueprint.__mro__

In [ ]:
dd = MyNewBlueprintDataset(data=dict(number=(1234,), text=['hello']))
dd._model
# dd.data['number']
# dd.__fields__

In [ ]:
A = Blueprint[list]

In [ ]:
B = Blueprint[str]

In [ ]:
hasattr(A(), 'mod')
A.mod
B().mod

In [ ]:
class AA:
    # @classmethod
    def __init_subclass__(cls, x: int, **kwargs):
        print(f'Initializing subclass {cls.__name__} with x={x}')
        return super().__init_subclass__(**kwargs)

    def __init__(self, a):
        self.a = a

In [ ]:
class BB(AA, x=3):
    def __init__(self, a, b):
        super().__init__(a)
        self.b = b

In [ ]:
class QuestBase:
    # this is implicitly a @classmethod (see below for motivation)
    def __init_subclass__(cls, swallow, **kwargs):
        cls.swallow = swallow
        super().__init_subclass__(**kwargs)

In [ ]:
class Quest(QuestBase, swallow="african"):
    pass

In [ ]:
Quest.swallow


In [ ]:
class MyBlueprint(TypedDict, total=False):
    number: int
    text: str

In [ ]:
from collections import UserDict


class MyModel(BaseModel, UserDict):
    data: MyBlueprint = Field(default_factory=MyBlueprint)

    class Config:
        validate_assignment = True
        arbitrary_types_allowed = True

In [ ]:
a = MyModel(data=dict(number='1234', text=2))

In [ ]:
a

In [ ]:
a

In [ ]:
asd.val

In [ ]:
A = TypedDict('A', {'number': int, 'text': str})

In [ ]:
A()

In [ ]:
BaseModel.validate(asd)